### Installing the libraries

In [ ]:
pip install nltk textblob  bs4 re


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
import pandas as pd

### Loading data

In [ ]:
from google.colab import files
upload=files.upload()

Saving IMDB Dataset.csv to IMDB Dataset.csv


In [ ]:
df=pd.read_csv('IMDB Dataset.csv')
df.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt


In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

### Preprocess the text

In [ ]:
## Preprocess the text
### Lowercase the text
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
def clean_text(text):
    # 1. Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # 2. Lowercase
    text = text.lower()

    # 3. Remove special characters and numbers
    text= re.sub('\[[^]]*\]', '',text)
    text = re.sub(r'[^a-z\s]', '', text)

    # 4. Tokenize
    words = nltk.word_tokenize(text)

    # 5. Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # 6. Lemmatizing (you can also use stemming instead)
    lemmatizer = WordNetLemmatizer()

    words = [lemmatizer.lemmatize(word) for word in words]

    # Final cleaned text
    return " ".join(words)


In [ ]:
df['process_txt']=df['review'].apply(clean_text)

In [ ]:
df.head(5)

,review,sentiment,process_txt
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode you...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically there family little boy jake think t...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...


### option if want to get score for sentiment


In [ ]:
# def get_sentiment(score):
#     if score > 0.1:
#         return 'Positive'
#     elif score < -0.1:
#         return 'Negative'
#     else:
#         return 'Neutral'

# def sentiment(data):
#     data['sentiment_score']=data['clean_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
#     data['sentiment']=data['sentiment_score'].apply( lambda x: get_sentiment(x))
#     return  data.drop('sentiment_score',axis=1)





### labelling categorical value

In [ ]:
# labelling the categorical value
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [ ]:
df['label']=encoder.fit_transform(df['sentiment'])

In [ ]:
df.head(5)

,review,sentiment,process_txt,label
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode you...,1
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...,1
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...,1
3,Basically there's a family where a little boy ...,negative,basically there family little boy jake think t...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...,1


In [ ]:
# Handling the imbalance dataset
# from sklearn.utils import resample

# # Combine into a single DataFrame
# positive = df_2[df_2.sentiment == 'Positive']
# neutral = df_2[df_2.sentiment == 'Neutral']
# negative = df_2[df_2.sentiment == 'Negative']

# # Downsample majority classes to match negative class
# positive_down = resample(positive,
#                          replace=False,     # sample without replacement
#                          n_samples=len(negative),
#                          random_state=42)

# neutral_down = resample(neutral,
#                         replace=False,
#                         n_samples=len(negative),
#                         random_state=42)

# # Combine balanced dataset
# df_balanced = pd.concat([positive_down, neutral_down, negative])

In [ ]:
X=df['process_txt']
Y=df['label']

### Model fitting

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features=5000)
X_tfidf = vectorizer.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y, test_size=0.2, random_state=40)

In [ ]:
from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(max_iter=1000)
# model.fit(X_train, y_train)

### Metric check

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.883

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.88      0.88      5100
           1       0.87      0.89      0.88      4900

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC,SVC

In [ ]:
# models = {
#     "Naive Bayes": MultinomialNB(),
#     "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
#     "SVM": SVC(kernel='linear')
# }

# # Train and evaluate
# results = {}

# for name, model in models.items():
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     acc = accuracy_score(y_test, y_pred)
#     results[name] = acc
#     print(f"\nModel: {name}")
#     print(f"Accuracy: {acc:.2f}")
#     print("Classification Report:")
#     print(classification_report(y_test, y_pred))

In [ ]:
def predict_sentiment(text):
    # Preprocess the input text
    cleaned_text = clean_text(text)
    vec=vectorizer.transform([cleaned_text])
    # Make a prediction
    prediction = model.predict(vec)
    return encoder.inverse_transform([prediction[0]])

In [ ]:
test=predict_sentiment(f"considering actor i dont think it wast his best work could have done better not that hook.")
print(test[0])

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1,100],        # Regularization strength
    'penalty': ['l1', 'l2'],             # Regularization penalty
    'solver': ['liblinear', 'saga']      # Solvers that support both penalties
}
logreg=LogisticRegression(max_iter=100)
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which 

Best Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score: 0.887425


In [ ]:
best_model=grid_search.best_estimator_

In [ ]:
import joblib

In [ ]:
joblib.dump(best_model, "best_model.pkl")

['best_model.pkl']

In [ ]:
# Save vectorizer
joblib.dump(vectorizer, "vectorizer.pkl")

# Save label encoder
joblib.dump(encoder, "encoder.pkl")

['encoder.pkl']

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_tfidf = vectorizer.fit_transform(df['process_txt'])  # FIT happens here

# 2. Save AFTER fit
import joblib
joblib.dump(vectorizer, "vectorizer.pkl")
from google.colab import files
files.download("vectorizer.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>